# **SISTEMAS DE RECOMENDACIÓN**

In [804]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [805]:
df = pd.read_csv('../data/raw/adult-census-income.csv')
df

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,Private,310152,Some-college,10,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,United-States,<=50K
32557,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32558,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32559,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K


In [806]:
# Estandarizar nombres de columnas
df.columns = df.columns.str.strip().str.lower().str.replace("-", ".")

In [807]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education.num   32561 non-null  int64 
 5   marital.status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital.gain    32561 non-null  int64 
 11  capital.loss    32561 non-null  int64 
 12  hours.per.week  32561 non-null  int64 
 13  native.country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


## 2. Preprocesamiento de datos. Haz la limpieza de datos nulos o mal codificados, la transformación de variables categóricas, normaliza las variables numéricas.

In [808]:
# Limpieza de nulos y valores mal codificados:
df = df.replace("?", pd.NA)
df = df.dropna().drop_duplicates()
df.shape

(30139, 15)

In [809]:
# Codificación de variables categóricas:
df = pd.get_dummies(df, drop_first=True)
df

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,native.country_Puerto-Rico,native.country_Scotland,native.country_South,native.country_Taiwan,native.country_Thailand,native.country_Trinadad&Tobago,native.country_United-States,native.country_Vietnam,native.country_Yugoslavia,income_>50K
1,82,132870,9,0,4356,18,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
3,54,140359,4,0,3900,40,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
4,41,264663,10,0,3900,40,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
5,34,216864,9,0,3770,45,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
6,38,150601,6,0,3770,40,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,310152,10,0,0,40,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
32557,27,257302,12,0,0,38,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
32558,40,154374,9,0,0,40,False,True,False,False,...,False,False,False,False,False,False,True,False,False,True
32559,58,151910,9,0,0,40,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False


### Aqui convierto columnas categóricas (como "sexo", "ocupación", "estado civil", etc.) en múltiples columnas binarias (0 o 1), una por cada categoría distinta que aparece en esas columnas.
### Por eso se pasa de 15 columnas a 97.

In [810]:
# Normalización de variables numéricas.
numericas = ['age', 'hours.per.week', "capital.gain", "capital.loss"]
scaler = MinMaxScaler()
df[numericas] = scaler.fit_transform(df[numericas])
df

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,native.country_Puerto-Rico,native.country_Scotland,native.country_South,native.country_Taiwan,native.country_Thailand,native.country_Trinadad&Tobago,native.country_United-States,native.country_Vietnam,native.country_Yugoslavia,income_>50K
1,0.890411,132870,9,0.0,1.000000,0.173469,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
3,0.506849,140359,4,0.0,0.895317,0.397959,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
4,0.328767,264663,10,0.0,0.895317,0.397959,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
5,0.232877,216864,9,0.0,0.865473,0.448980,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
6,0.287671,150601,6,0.0,0.865473,0.397959,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,0.068493,310152,10,0.0,0.000000,0.397959,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
32557,0.136986,257302,12,0.0,0.000000,0.377551,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
32558,0.315068,154374,9,0.0,0.000000,0.397959,False,True,False,False,...,False,False,False,False,False,False,True,False,False,True
32559,0.561644,151910,9,0.0,0.000000,0.397959,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False


### Escalamos porque vamos a usar metodos basados en similitudes.

In [811]:
# Revision de columnas poco utiles.
ceros = (df == 0).sum() / len(df)
columnas_poco_utiles = ceros[ceros > 0.9]
columnas_poco_utiles

capital.gain                      0.915790
capital.loss                      0.952653
workclass_Local-gov               0.931418
workclass_Self-emp-inc            0.964365
workclass_Self-emp-not-inc        0.917117
                                    ...   
native.country_Taiwan             0.998606
native.country_Thailand           0.999436
native.country_Trinadad&Tobago    0.999403
native.country_Vietnam            0.997877
native.country_Yugoslavia         0.999469
Length: 75, dtype: float64

In [812]:
# Eliminar columnas que tiene mas del 99% de ceros.
umbral = 0.99
porcentaje_ceros = (df == 0).sum() / len(df)
columnas_a_quitar = porcentaje_ceros[porcentaje_ceros > umbral].index
df = df.drop(columns=columnas_a_quitar)
df

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Asian-Pac-Islander,race_Black,race_White,sex_Male,native.country_Mexico,native.country_United-States,income_>50K
1,0.890411,132870,9,0.0,1.000000,0.173469,False,True,False,False,...,False,False,False,False,False,True,False,False,True,False
3,0.506849,140359,4,0.0,0.895317,0.397959,False,True,False,False,...,False,True,False,False,False,True,False,False,True,False
4,0.328767,264663,10,0.0,0.895317,0.397959,False,True,False,False,...,True,False,False,False,False,True,False,False,True,False
5,0.232877,216864,9,0.0,0.865473,0.448980,False,True,False,False,...,False,True,False,False,False,True,False,False,True,False
6,0.287671,150601,6,0.0,0.865473,0.397959,False,True,False,False,...,False,True,False,False,False,True,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,0.068493,310152,10,0.0,0.000000,0.397959,False,True,False,False,...,False,False,False,False,False,True,True,False,True,False
32557,0.136986,257302,12,0.0,0.000000,0.377551,False,True,False,False,...,False,False,True,False,False,True,False,False,True,False
32558,0.315068,154374,9,0.0,0.000000,0.397959,False,True,False,False,...,False,False,False,False,False,True,True,False,True,True
32559,0.561644,151910,9,0.0,0.000000,0.397959,False,True,False,False,...,False,True,False,False,False,True,False,False,True,False


### Eliminamos columnas que tienen más del 99% de ceros

In [813]:
# Ojo en "age" con los ceros.
ceros = (df == 0).sum()
ceros[ceros > 0]

age                                       328
capital.gain                            27601
capital.loss                            28712
hours.per.week                              7
workclass_Local-gov                     28072
workclass_Private                        7875
workclass_Self-emp-inc                  29065
workclass_Self-emp-not-inc              27641
workclass_State-gov                     28860
education_11th                          29091
education_12th                          29762
education_7th-8th                       29583
education_9th                           29684
education_Assoc-acdm                    29131
education_Assoc-voc                     28832
education_Bachelors                     25097
education_Doctorate                     29764
education_HS-grad                       20305
education_Masters                       28513
education_Prof-school                   29597
education_Some-college                  23470
marital.status_Married-civ-spouse 

In [814]:
df = df[df["age"] != 0]
ceros = (df == 0).sum()
ceros[ceros > 0]

capital.gain                            27281
capital.loss                            28394
hours.per.week                              7
workclass_Local-gov                     27758
workclass_Private                        7845
workclass_Self-emp-inc                  28744
workclass_Self-emp-not-inc              27319
workclass_State-gov                     28534
education_11th                          28917
education_12th                          29466
education_7th-8th                       29258
education_9th                           29379
education_Assoc-acdm                    28803
education_Assoc-voc                     28504
education_Bachelors                     24769
education_Doctorate                     29436
education_HS-grad                       19984
education_Masters                       28185
education_Prof-school                   29269
education_Some-college                  23144
marital.status_Married-civ-spouse       15754
marital.status_Married-spouse-abse

### Eliminamos los ceros de "age" para quitar ruido.

# 3. Definición del Sistema de Recomendación

**¿Qué se quiere recomendar?**

Se quieren recomendar trayectorias profesionales realistas para personas con características socioeconómicas específicas. Específicamente:

- Ocupaciones predominantes

- Niveles educativos frecuentes

- Rango de horas de trabajo típico

- Combinaciones que suelen asociarse a un ingreso >50K

El objetivo es que, dado un perfil simulado, el sistema diga:

> “Personas como tú tienden a trabajar en X ocupación, con Y nivel educativo, y ganan más de 50K.”

**¿Quién es el "usuario"?**

Un perfil simulado que representa a una persona hipotética. Puede construirse manualmente o generarse automáticamente, e incluir variables como:

- Edad

- Nivel educativo (education.num)

- Jornada laboral (hours.per.week)

- Rol deseado (occupation_X)

- Sexo, país, estado civil (si se quiere afinar más)

Es decir, un vector con las mismas dimensiones que tus datos (X), pero personalizado.

**¿Qué variables definen al usuario?**

Tu perfil de usuario estará definido por un subconjunto de estas variables:
- Educacion : education.num o education_Bachelors
- Jornada laboral : hour.perweek
- Ocupacion deseada : ocupacion_Exec-managerial
- Sexo: sex_Male o sex_Female
- País de origen : native.country_*
- Estado civil : marital.status_*

# 4. Construye el sistema de recomendación.

## Filtrado basado en contenido.

In [815]:
# Separar etiquetas
target_col = [col for col in df.columns if "income" in col][0]
y = df[target_col]
X = df.drop(columns=[target_col])

In [816]:
# Filtrado: solo personas con ingreso >50K
ricos = X[y == 1].copy()
ricos = ricos[ricos[[c for c in ricos.columns if "occupation" in c]].sum(axis=1) > 0]
ricos = ricos.reindex(columns=X.columns, fill_value=0)

In [817]:
# Simulación del perfil
perfil_usuario = X.mean().copy()
perfil_usuario["education.num"] = 13              # Universitario
perfil_usuario["hours.per.week"] = 0.5            # 50 horas normalizadas
perfil_usuario["occupation_Exec-managerial"] = 1  # Desea puesto ejecutivo
perfil_usuario = perfil_usuario.reindex(X.columns, fill_value=0)

In [818]:
# Similaridad coseno
similitudes = cosine_similarity([perfil_usuario], ricos)[0]
top_idx = np.argsort(similitudes)[-10:]

In [819]:
# Top 10 recomendaciones
recomendaciones = ricos.iloc[top_idx].copy()

In [820]:
# Taducimos los vectores codificados en ocupaciones comprensibles.
def extraer_ocupacion(fila): # función que recibe una fila del DataFrame (una persona recomendada).
    for col in fila.index: # recorre todos los nombres de columnas de esa fila.
        if "occupation" in col and fila[col] in [1, True]: # Busca las columnas cuyo nombre contiene "occupation" y que tengan un valor activo (1 o True).
            return col.replace("occupation_", "").replace("occupation.", "")# Cuando encuentra la primera columna que cumple eso, devuelve su nombre limpio.
    return "No especificado" 

recomendaciones = recomendaciones.copy()
recomendaciones["ocupacion"] = recomendaciones.apply(extraer_ocupacion, axis=1)
recomendaciones[["education.num", "hours.per.week", "ocupacion"]]

,education.num,hours.per.week,ocupacion
2773,13,0.500000,Exec-managerial
1981,14,0.653061,Exec-managerial
28762,14,0.551020,Exec-managerial
2072,14,0.397959,Exec-managerial
15437,14,0.602041,Exec-managerial
25248,14,0.653061,Exec-managerial
29247,14,0.500000,Exec-managerial
9853,14,0.500000,Exec-managerial
19165,14,0.448980,Exec-managerial
13363,14,0.500000,Exec-managerial


**Las 10 personas con ingresos >50K que son más similares:**

- Tienen educación universitaria completa o superior (13–14 en education.num)

- Trabajan entre 40 y 65 horas semanales ( columna hours.per.week)

- Puestos ejecutivos, todos tienen: “Exec-managerial”.

Son personas que tienen alto nivel educativo, trabajan duro y ocupan puestos ejecutivos bien remunerados.

# 5. Pruebas con casos simulados. Construye perfiles simulados de usuarios hipotéticos y fijate qué trayectorias (educación, ocupación, etc.) les recomendaría el sistema para mejorar su ingreso estimado.

In [821]:
# Simulamos perfiles de usuarios
def crear_perfil(educ_num, horas_norm, ocupacion=None):
    perfil = X.mean().copy()
    perfil["education.num"] = educ_num
    perfil["hours.per.week"] = horas_norm
    if ocupacion and f"occupation_{ocupacion}" in X.columns:
        perfil[f"occupation_{ocupacion}"] = 1
    return perfil.reindex(X.columns, fill_value=0)

In [822]:
# Recomendar trayectorias.
def recomendar_trayectorias(perfil_usuario, base=ricos, top_n=10):
    similitudes = cosine_similarity([perfil_usuario], base)[0]
    top_idx = np.argsort(similitudes)[-top_n:]
    recomendaciones = base.iloc[top_idx].copy()

    def extraer_ocupacion(fila):
        for col in fila.index:
            if "occupation" in col and fila[col] in [1, True]:
                return col.replace("occupation_", "").replace("occupation.", "")
        return "No especificado"

    recomendaciones["ocupacion"] = recomendaciones.apply(extraer_ocupacion, axis=1)
    return recomendaciones[["education.num", "hours.per.week", "ocupacion"]]

# CASOS SIMULADOS

In [823]:
# Usuario de 25 años, secundario completo, trabaja medio tiempo
# education.num = 9 (HS-graduate), horas normalizadas 0.25 (20h/semana)
perfil_A = crear_perfil(educ_num=9, horas_norm=0.25)
recomendaciones_A = recomendar_trayectorias(perfil_A)
recomendaciones_A

,education.num,hours.per.week,ocupacion
7116,13,0.500000,Exec-managerial
27818,13,0.448980,Sales
17368,13,0.500000,Sales
12377,13,0.397959,Prof-specialty
3719,13,0.500000,Exec-managerial
12901,13,0.500000,Exec-managerial
10715,13,0.448980,Craft-repair
13006,13,0.500000,Exec-managerial
13882,13,0.397959,Prof-specialty
19348,13,0.397959,Prof-specialty


- Formación significativamente más alta (todas las recomendaciones tienen education.num = 13, equivalente a licenciatura)

- Mayor carga horaria (entre ~40 y 50 horas semanales)

- Ocupaciones mejor posicionadas: ejecutivos, profesionales especializados, ventas

In [824]:
# Usuario universitario, jornada completa, desea ocupación técnica
perfil_B = crear_perfil(educ_num=13, horas_norm=0.5, ocupacion="Tech-support")
recomendaciones_B = recomendar_trayectorias(perfil_B)
recomendaciones_B

,education.num,hours.per.week,ocupacion
11171,16,0.602041,Prof-specialty
21791,12,0.448980,Tech-support
31761,13,0.602041,Tech-support
1860,13,0.397959,Tech-support
12989,12,0.397959,Tech-support
28576,13,0.479592,Tech-support
408,14,0.193878,Tech-support
15366,13,0.397959,Tech-support
16935,13,0.448980,Tech-support
1813,13,0.397959,Tech-support


- Formación sólida: mayoría con education.num = 13 o más (equivale a licenciatura, e incluso hay uno con doctorado 16).

- Carga horaria estable: en general, ~40 a 60 horas semanales (normalizado).

- Alta concordancia con el rol deseado: 9 de 10 recomendaciones están en Tech-support.

In [825]:
#Usuario sin estudios superiores, pero con alta dedicación.
perfil_C = crear_perfil(educ_num=9, horas_norm=0.8)
recomendaciones_C = recomendar_trayectorias(perfil_C)
recomendaciones_C

,education.num,hours.per.week,ocupacion
19348,13,0.397959,Prof-specialty
25017,13,0.500000,Craft-repair
25758,13,0.500000,Exec-managerial
7116,13,0.500000,Exec-managerial
10715,13,0.448980,Craft-repair
17368,13,0.500000,Sales
3719,13,0.500000,Exec-managerial
15769,13,0.551020,Exec-managerial
13006,13,0.500000,Exec-managerial
12901,13,0.500000,Exec-managerial


- Educación terciaria o universitaria (education.num ≈ 13)

- Carga horaria media-alta

- Ocupaciones de responsabilidad o con alta especialización